placeholder

In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import xskillscore as xs
import dask.array as da
from dask.distributed import Client

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:35657 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 6 Memory: 16.73 GB


Due to the success of your previous forecast (and verification using xskillscore!) the company you work for has expanded... 400 stores each with 3000 products

In [4]:
nstores = 400
nskus = 3000
nforecasts = nstores * nskus
print(f"That's {nforecasts:,d} forecasts!")

stores = np.arange(nstores)
skus = np.arange(nskus)

That's 1,200,000 forecasts!


In [5]:
dates = pd.date_range("1/1/2021", "1/5/2021", freq="D")

In [6]:
data = da.random.randint(10, size=(len(dates), len(stores), len(skus)))
data

dask.array<randint, shape=(5, 400, 3000), dtype=int64, chunksize=(5, 400, 3000), chunktype=numpy.ndarray>

In [7]:
y = xr.DataArray(data, coords=[dates, stores, skus], dims=["DATE", "STORE", "SKU"])
y

<xarray.DataArray 'randint-bd7610cfcc8b874b64e81617f8e41b8d' (DATE: 5, STORE: 400, SKU: 3000)>
dask.array<randint, shape=(5, 400, 3000), dtype=int64, chunksize=(5, 400, 3000), chunktype=numpy.ndarray>
Coordinates:
  * DATE     (DATE) datetime64[ns] 2021-01-01 2021-01-02 ... 2021-01-05
  * STORE    (STORE) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
  * SKU      (SKU) int64 0 1 2 3 4 5 6 7 ... 2993 2994 2995 2996 2997 2998 2999

In [8]:
noise = da.random.uniform(-1, 1, size=(len(dates), len(stores), len(skus)))
yhat = y + (y * noise)
yhat

<xarray.DataArray (DATE: 5, STORE: 400, SKU: 3000)>
dask.array<add, shape=(5, 400, 3000), dtype=float64, chunksize=(5, 400, 3000), chunktype=numpy.ndarray>
Coordinates:
  * DATE     (DATE) datetime64[ns] 2021-01-01 2021-01-02 ... 2021-01-05
  * STORE    (STORE) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
  * SKU      (SKU) int64 0 1 2 3 4 5 6 7 ... 2993 2994 2995 2996 2997 2998 2999

Set chunk to one on the dimension to reduce over...

In [9]:
print(xs.rmse(y, yhat, 'DATE').compute())

<xarray.DataArray (STORE: 400, SKU: 3000)>
array([[1.47384281, 1.74887684, 2.10411806, ..., 1.13449828, 3.00613651,
        3.28063828],
       [2.84047935, 1.83978778, 1.54562646, ..., 4.06570152, 2.67096194,
        1.97370806],
       [1.95833744, 2.28858981, 0.65809608, ..., 3.94344424, 2.95879069,
        3.61857448],
       ...,
       [2.58347277, 2.93381188, 2.35829552, ..., 3.6992895 , 3.75497599,
        2.34951225],
       [2.59031842, 4.90280829, 3.72707129, ..., 2.8933763 , 4.75206652,
        2.24969129],
       [2.59321234, 2.85580076, 4.99678729, ..., 1.96431186, 5.52325469,
        2.42215324]])
Coordinates:
  * STORE    (STORE) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
  * SKU      (SKU) int64 0 1 2 3 4 5 6 7 ... 2993 2994 2995 2996 2997 2998 2999


Next year you are expected to grow again due to some great forecasts etc...

In [11]:
nstores = 4000
nskus = 30000
nforecasts = nstores * nskus
print(f"That's {nforecasts:,d} forecasts!")

stores = np.arange(nstores)
skus = np.arange(nskus)

data = da.random.randint(10, size=(len(dates), len(stores), len(skus)))
data

That's 120,000,000 forecasts!


dask.array<randint, shape=(5, 4000, 30000), dtype=int64, chunksize=(5, 1000, 1500), chunktype=numpy.ndarray>

In [12]:
y = xr.DataArray(data, coords=[dates, stores, skus], dims=["DATE", "STORE", "SKU"])

noise = da.random.uniform(-1, 1, size=(len(dates), len(stores), len(skus)))
yhat = y + (y * noise)
yhat

<xarray.DataArray (DATE: 5, STORE: 4000, SKU: 30000)>
dask.array<add, shape=(5, 4000, 30000), dtype=float64, chunksize=(5, 1000, 1500), chunktype=numpy.ndarray>
Coordinates:
  * DATE     (DATE) datetime64[ns] 2021-01-01 2021-01-02 ... 2021-01-05
  * STORE    (STORE) int64 0 1 2 3 4 5 6 ... 3993 3994 3995 3996 3997 3998 3999
  * SKU      (SKU) int64 0 1 2 3 4 5 6 ... 29994 29995 29996 29997 29998 29999

In [13]:
print(xs.rmse(y, yhat, 'DATE').compute())

<xarray.DataArray (STORE: 4000, SKU: 30000)>
array([[3.3978793 , 3.52061817, 4.4017451 , ..., 5.74607406, 2.27697634,
        2.61495648],
       [5.32447308, 2.56624199, 2.30826983, ..., 2.44459472, 3.2885685 ,
        3.05550171],
       [3.85131871, 4.1388258 , 1.76673018, ..., 2.05920206, 3.46622161,
        2.48781756],
       ...,
       [1.2467093 , 0.79060679, 3.0253551 , ..., 2.19662638, 4.76962681,
        4.77508544],
       [3.0192147 , 2.01889082, 4.11861269, ..., 1.15534179, 3.06923749,
        2.42122111],
       [4.04498502, 3.59812936, 3.53601828, ..., 2.98678067, 2.70577774,
        1.81942665]])
Coordinates:
  * STORE    (STORE) int64 0 1 2 3 4 5 6 ... 3993 3994 3995 3996 3997 3998 3999
  * SKU      (SKU) int64 0 1 2 3 4 5 6 ... 29994 29995 29996 29997 29998 29999
